# My code for computing minbounce

Load code for parking functions

In [1]:
# Load the parking function class
load("parking_functions.py")

Create methods for Dyck paths and minbounce

In [2]:
# Function that generates all possible (n,nk)-Dyck paths
def genfun_Dyck(n,k):
    # The list has +k for a NORTH step and -1 for an EAST step
    list_of_paths = [[k]]
    for length in range(n*k + n - 1):                               # Iterate on the lenght of the lists, add one step
        new_list = []
        for path in list_of_paths:
            if path.count(k) < n:
                new_list = new_list + [path + [k]]     # Add an NORTH step
            if sum(path) > 0:
                new_list = new_list + [path + [-1]]    # Add an EAST step
        list_of_paths = new_list
    # Translate all k's to +1
    transl_list = []
    for path in list_of_paths:
        for i in range(n*k + n):
            if path[i] == k:
                path[i] = 0
        area_path = [n*k + sum(path[0:i]) for i in range(n*k + n) if path[i] == 0]
        transl_list = transl_list + [area_path]
    # Translate to area
    return transl_list

# Compute the bounce for k=1
def bounce(path):
    n = len(path)
    bounce = 0
    x = 0
    iterat = 0
    while x < n:
        x_new = sum([1 for v in path if v >= n - x])
        bounce += iterat*(x_new-x)
        x = x_new
        iterat += 1
    return bounce

# Checks if a path is Dyck
def ok_path(n,k,path):
    for i in range(n-1):                        # Checks decreasing
        if path[i] < path[i+1]:
            return False
    for i in range(n):                          # Checks values on each row
        if path[i] > n*k or path[i] < k*(n-i):      
            return False
    return True

# Compute all decompositions recursively to get min_bounce
def min_bounce(n,k):
    # The result is given by:
    #       dictionary indexed on n,nk - dyck paths (TUPLES)
    #       associated to a list where:
    #           [0]:    minbounce value
    #           [1]:    list of all possible decompositions
    Dk = genfun_Dyck(n,k)       # Compute all dyck paths
    result = {}
    if k == 1:                      # Base      case k = 1
        for path in Dk:
            result = result | { tuple(path) : [ bounce(path), [[tuple(path)]] ] }
    else:                           # Recursive case k > 1
        res_one = min_bounce(n,1)           # Compute k = 1
        res_prev = min_bounce(n,k-1)        # Compute previous
        Dk_prev = list(res_prev.keys())
        print("\tmin_bounce for k = {}".format(k))
        for path in Dk:                     # Loop through all new paths
            minbounce = binomial(n,2)*k
            possible_dec = []
            for prev in Dk_prev:            # Loop through old paths
                new_part = [path[i] - prev[i] for i in range(n)]
                if ok_path(n,1,new_part):                                       # If it is a valid decomposition...
                    if res_prev[prev][0] + res_one[tuple(new_part)][0] < minbounce:        #   ...and new minbounce
                        possible_dec = [sorted(dec + [tuple(new_part)]) for dec in res_prev[prev][1]]
                        minbounce = res_prev[prev][0] + res_one[tuple(new_part)][0]
                    else:
                        if res_prev[prev][0] + res_one[tuple(new_part)][0] == minbounce:   #   ...and same minbounce
                            new_dec_list = [sorted(dec + [tuple(new_part)]) for dec in res_prev[prev][1]]    # compute new decompositions
                            for new_dec in new_dec_list:                                                     # remove redundants
                                if new_dec not in possible_dec:
                                    possible_dec = possible_dec + [new_dec]
            result = result | {tuple(path) : [minbounce, possible_dec]}
    return result

Create methods to decompose pmaj word

In [3]:
# Contributes of maj
def word_maj(n,k,word):
    if k == 0:
        return {i:0 for i in range(1,n+1)}
    contributes = {}
    ascends = [word[i] <= word[i+1] for i in range(n*k - 1)] + [False]
    
    for i in range(1,n+1):
        contr_i = sum(ascends[0:word.index(i)])
        contributes = contributes | {i:contr_i}
    return contributes

# Possible decomposition of wpmaj
def decomp_wpmaj(n,k,wpmaj):
    decomp = []
    contrib = word_maj(n,k,wpmaj)
    for i in range(k):
        #print(contrib)
        if k-i == 1:
            new_word = []
        else:
            new_word = [wpmaj[j] for j in range(len(wpmaj)) if wpmaj[j] in wpmaj[0:j]]     # Delete first occurrencies
        #print(new_word)
        new_contrib = word_maj(n,k-i-1,new_word)
        part_decomp = {i:contrib[i]-new_contrib[i] for i in range(1,n+1)}   # Compute partial contribution
        decomp = decomp + [part_decomp]                                     # Add new partial contribution
        contrib = new_contrib                                               # Update contributes
        wpmaj = new_word                                                    # Update word
    return decomp

## Test 1
See if pmaj contributes are preserved

In [4]:
n = 5
k = 5

label = [i+1 for i in range(n)]         # Define the standard labelling for Dyck paths

print("Generating the paths...")
DPk = genfun_Dyck(n,k)                  # Compute all nk Dyck
DPsq = genfun_Dyck(n,1)                 # Compute all n Dyck

print("Computing minbounce...")
decompositions = min_bounce(n,k)        # Compute min_bounce and decompositions

print("Checking decompositions...")
for path in DPk:                        # Loop over all paths 
    pf_path = ParkFunc(n,n*k,w_area=path,w_label=label)                       # Convert to parking function
    [pmaj_bounce, contributes, wpmaj] = pf_path.pmaj(infos=True)            # Compute pmaj (i.e. k-bounce)

    #print("Testing Dyck path: {}".format(path))
    check = False
    for decomp in decompositions[tuple(path)][1]:                              # Run over possible minimal decompositions and look if contributes sum well...
        contr_list = []             
        for factor in decomp:                                               # Compute contributes of decomposition
            contr_factor = []
            pf_factor = ParkFunc(n,n,w_area=list(factor),w_label=label)
            [pmaj_bounce, contr_factor, wpmaj_factor] = pf_factor.pmaj(infos=True)
            contr_list = contr_list + [contr_factor]
        contr_decomp = {i+1:sum([c[i+1] for c in contr_list]) for i in range(n)}# Sum contributes
        if contributes == contr_decomp:                                             # Contributes MATCH!
            #print("\tContributes match using decomposition {}".format(decomp))
            check = True
        else:
            print("\tDecomposition {} does not work:\n\t\tcontributes = {}\n\t\tcontr_decomp = {}".format(decomp, contributes, contr_decomp))

    if not check:                                                                   # No decomposition match...
            print("\tNO suitable decomposition found...")




Generating the paths...
Computing minbounce...
	min_bounce for k = 2
	min_bounce for k = 3
	min_bounce for k = 4
	min_bounce for k = 5
Checking decompositions...


## Test 2
Test the decomposition of wpmaj

In [5]:
n = 4
k = 3

label = [i+1 for i in range(n)]         # Define the standard labelling for Dyck paths

DPk = genfun_Dyck(n,k)                  # Compute all nk Dyck
DPsq = genfun_Dyck(n,1)                 # Compute all n Dyck

decompositions = min_bounce(n,k)        # Compute min_bounce and decompositions

for path in DPk:                        # Loop over all paths 
    pf_path = ParkFunc(n,n*k,w_area=path,w_label=label)                         # Convert to parking function
    [pmaj_bounce, contributes, wpmaj] = pf_path.pmaj(infos=True)                # Compute pmaj (i.e. k-bounce)

    print("\nTesting Dyck path: {}".format(path))
    check = False
    for decomp in decompositions[tuple(path)][1]:                               # Run over possible minimal decompositions and look if contributes sum well...
        contr_list = []             
        for factor in decomp:                                                   # Compute contributes of decomposition
            contr_factor = []
            pf_factor = ParkFunc(n,n,w_area=list(factor),w_label=label)
            [pmaj_factor, contr_factor, wpmaj_factor] = pf_factor.pmaj(infos=True)
            contr_list = contr_list + [contr_factor]
        
        wpmaj_list = decomp_wpmaj(n,k,wpmaj)            # Compute the decomposition from wpmaj
        
        wpmaj_sort = sorted([sorted([[i,part[i]] for i in part.keys()]) for part in wpmaj_list])
        contr_sort = sorted([sorted([[i,part[i]] for i in part.keys()]) for part in contr_list])

        if wpmaj_sort == contr_sort:                    # Check if contribute decompositions coincide
            print("\tDecompositions coincide!")
            check = True
        else:
            print("\tNOPE minbounce = {}\t pmaj = {}\t wpmaj = {}\n\t\tminbou = {}\n\t\tdecomp = {}".format(decompositions[tuple(path)][0],pmaj_bounce,wpmaj,contr_list,wpmaj_list))
    
    if not check:
        print("\t\t\tNO good match...")




	min_bounce for k = 2
	min_bounce for k = 3

Testing Dyck path: [12, 12, 12, 12]
	Decompositions coincide!

Testing Dyck path: [12, 12, 12, 11]
	Decompositions coincide!

Testing Dyck path: [12, 12, 12, 10]
	Decompositions coincide!

Testing Dyck path: [12, 12, 12, 9]
	Decompositions coincide!

Testing Dyck path: [12, 12, 12, 8]
	Decompositions coincide!
	Decompositions coincide!

Testing Dyck path: [12, 12, 12, 7]
	Decompositions coincide!

Testing Dyck path: [12, 12, 12, 6]
	Decompositions coincide!

Testing Dyck path: [12, 12, 12, 5]
	Decompositions coincide!
	Decompositions coincide!

Testing Dyck path: [12, 12, 12, 4]
	Decompositions coincide!

Testing Dyck path: [12, 12, 12, 3]
	Decompositions coincide!

Testing Dyck path: [12, 12, 11, 11]
	Decompositions coincide!

Testing Dyck path: [12, 12, 11, 10]
	Decompositions coincide!

Testing Dyck path: [12, 12, 11, 9]
	Decompositions coincide!
	NOPE minbounce = 3	 pmaj = 3	 wpmaj = [2, 1, 3, 2, 1, 4, 3, 2, 1, 4, 3, 4]
		minbou = [{2: 0

In [6]:
word_maj(4,2,[1, 1, 2, 2, 3, 3, 4, 4])

{1: 0, 2: 2, 3: 4, 4: 6}

In [7]:
X = [1,2,3,4,5,6]
X[1:3]

[2, 3]

In [8]:
dic = {1:1,2:1}
sum(dic.values())

2

# Testing new pmaj

In [11]:
pf_test = ParkFunc(3,5,w_area=[15,12,10],w_label=[3,1,2])

pf_test.pmaj()

ValueError: min() iterable argument is empty